#### Get mapping of dungeon -> id

In their result jsons blizz uses interger dungeon ids. Get mapping of dungeon names to ids. 

Note that these ids are for the "challenge mode" of the dungeon. There is another argumen "map" with its own id, which refers to the overall location (regardless of difficulty). We don't care about the map id.

In [2]:
import requests
import importlib
import blizzard_api
import mysql
import mplus_db
import blizz_credentials
import pandas as pd

In [4]:
#grab access token
importlib.reload(blizz_credentials)
tokens_file_path = '.api_tokens'
authorization = blizz_credentials.Credentials(tokens_file_path)
access_token = authorization.access_token

In [9]:
#get m+ dungeon index
url = 'https://us.api.blizzard.com/data/wow/mythic-keystone/dungeon/index?namespace=dynamic-us&locale=en_US&access_token='
api_url = url + access_token
key_index_response = requests.get(api_url)

In [26]:
#these are ids for BFA dungeons
bfa_key_index = pd.DataFrame(key_index_response.json()['dungeons'])
bfa_key_index[['id', 'name']]

,id,name
0,244,Atal'Dazar
1,245,Freehold
2,246,Tol Dagor
3,247,The MOTHERLODE!!
4,248,Waycrest Manor
5,249,Kings' Rest
6,250,Temple of Sethraliss
7,251,The Underrot
8,252,Shrine of the Storm
9,353,Siege of Boralus


In [37]:
#there is no index call for Legion dungeons
#so here is a programmatic sweep of every id from 0 to 243

leader_board_url = ('https://us.api.blizzard.com/data/wow/'
                    'connected-realm/11/mythic-leaderboard/'
                    '{potential_id}/period/625?namespace=dynamic-us&'
                    'locale=en_US&access_token={access_token}')

valid_ids = []
for potential_id in range(0,244):
    url = leader_board_url.format(potential_id = potential_id, 
                                  access_token = access_token)
    response = requests.get(url)
    if response.status_code == 200:
        valid_ids.append(potential_id)
    print('\rtesting %s out of %s' % (potential_id, 244), end='')
print('')
print(valid_ids)

testing 243 out of 244
[197, 198, 199, 200, 206, 207, 208, 209, 210, 227, 233, 234, 239]


In [44]:
# Previous cell returned these valid ids
# valid_ids = [197, 198, 199, 200, 206, 207, 208, 209, 210, 227, 233, 234, 239]
legion_dungeon_id_map = []
for id_ in valid_ids:
    url = leader_board_url.format(potential_id = id_, 
                                  access_token = access_token)
    response = requests.get(url)
    dungeon_name = response.json()['map']['name']
    legion_dungeon_id_map.append([id_, dungeon_name])
print(legion_dungeon_id_map)

[[197, 'Eye of Azshara'], [198, 'Darkheart Thicket'], [199, 'Black Rook Hold'], [200, 'Halls of Valor'], [206, "Neltharion's Lair"], [207, 'Vault of the Wardens'], [208, 'Maw of Souls'], [209, 'The Arcway'], [210, 'Court of Stars'], [227, 'Return to Karazhan'], [233, 'Cathedral of Eternal Night'], [234, 'Return to Karazhan'], [239, 'Seat of the Triumvirate']]


In [47]:
legion_key_index = pd.DataFrame(legion_dungeon_id_map,
                                columns = ['id', 'name'])
legion_key_index

,id,name
0,197,Eye of Azshara
1,198,Darkheart Thicket
2,199,Black Rook Hold
3,200,Halls of Valor
4,206,Neltharion's Lair
5,207,Vault of the Wardens
6,208,Maw of Souls
7,209,The Arcway
8,210,Court of Stars
9,227,Return to Karazhan


In [50]:
#stack BFA and Legion keys together
#and push to MDB
bfa_legi_keys = pd.concat([legion_key_index, bfa_key_index[['id', 'name']]])
bfa_legi_keys

,id,name
0,197,Eye of Azshara
1,198,Darkheart Thicket
2,199,Black Rook Hold
3,200,Halls of Valor
4,206,Neltharion's Lair
5,207,Vault of the Wardens
6,208,Maw of Souls
7,209,The Arcway
8,210,Court of Stars
9,227,Return to Karazhan


In [62]:
insert_query = 'INSERT into dungeon VALUES '
values = []
for id_, name in bfa_legi_keys.values.tolist():
    value = '(%s, "%s")' % (id_, name)
    values.append(value)
values_string = ','.join(values)
insert_query += values_string
print(insert_query)

INSERT into dungeon VALUES (197, "Eye of Azshara"),(198, "Darkheart Thicket"),(199, "Black Rook Hold"),(200, "Halls of Valor"),(206, "Neltharion's Lair"),(207, "Vault of the Wardens"),(208, "Maw of Souls"),(209, "The Arcway"),(210, "Court of Stars"),(227, "Return to Karazhan"),(233, "Cathedral of Eternal Night"),(234, "Return to Karazhan"),(239, "Seat of the Triumvirate"),(244, "Atal'Dazar"),(245, "Freehold"),(246, "Tol Dagor"),(247, "The MOTHERLODE!!"),(248, "Waycrest Manor"),(249, "Kings' Rest"),(250, "Temple of Sethraliss"),(251, "The Underrot"),(252, "Shrine of the Storm"),(353, "Siege of Boralus"),(369, "Operation: Mechagon - Junkyard"),(370, "Operation: Mechagon - Workshop")


In [64]:
#grab list of realm ids from MDB
#conn.close()
mdb = mplus_db.MplusDB('.db_config')
conn = mdb.connect()
cursor = conn.cursor()
cursor.execute(insert_query)
conn.commit()
cursor.close()
conn.close()